# Set-up

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os


In [ ]:
DIR = '/content/drive/My Drive/ml-mini-projects/black-friday'

# Framing of the Problem

## Challenge:

The following dataset contains information from stores on a Black Friday, which sometimes refer to the Thanksgiving Friday in the US. You are tasked to build a model to predict the purchase amount of customers against various products.

**Evaluation Metric:** RMSE <br>
**Top scorer score:** 2372.0034421821

**Notes:** <br>
This challenge and dataset was sourced from [Analytics Vidhya](https://datahack.analyticsvidhya.com/contest/black-friday/?utm_source=facebook.com&utm_medium=social)


**Approach:** Regression Problem

# Taking Quick Glances

In [ ]:
df = pd.read_csv(os.path.join(DIR, 'train.csv'))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


**Observations:**
* It has a total of 550, 068 entries with 12 columns
   * Not so many features. Let's not do feature selection first
* Data types
   * User_ID, Product_ID 
      * not sure if this will help; try dropping
   * Gender, City Category as strings
      * hot encode this
   * Age, Stay In Current City Years are unexpectedly written as strings
      * Label Encoding - try converting to int or float (check unique values to decide)
   * Occupation is int (so already label encoded)
      * This may be placed for a reason. Let's try using this before hot encoding
   * Marital Status is already in its correct form
   * Product Category 1 and 2 are ints and 3 are floats.
      * can we convert Product Category 3 as int too?
      * also let's try to use this(label encoded) before doing one-hot
   * Purchase
      * does this represent the actual or in thousands or what? (no info to verify this)
* Nulls
   * Product Category 2 and 3 have many nulls


1. Let's first deal with the nulls

In [ ]:
pd.concat([pd.DataFrame(df.isna().sum(), columns=['count']),(pd.DataFrame((df.isna().sum()/df.shape[0])*100, columns=['percentage']))], axis=1)

,count,percentage
User_ID,0,0.000000
Product_ID,0,0.000000
Gender,0,0.000000
Age,0,0.000000
Occupation,0,0.000000
City_Category,0,0.000000
Stay_In_Current_City_Years,0,0.000000
Marital_Status,0,0.000000
Product_Category_1,0,0.000000
Product_Category_2,173638,31.566643


In [ ]:
# what if we drop Product_Category_2 only? How will that affect Product Category 3?
df_dum = df.dropna(subset=['Product_Category_2'])
df_dum.isna().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2                 0
Product_Category_3            209609
Purchase                           0
dtype: int64

**Possible approach for NaNs**
1. If we drop all the nulls,
   * we will only be left with 166,821 which is 30% of the original dataset.
2. If we drop only Product_Category_2, 
   * 173, 638 are deducted so it means, those that are null in Product_Category_2 are also null in Product_Category_3
3. Replace nulls with 0s
   * I'll be trying this approach



In [ ]:
df = df.fillna(0)

2. Are there duplicates? none.

In [ ]:
# are there duplicates?
df.duplicated().sum()

0

3. Convert Product_Category_3

In [ ]:
# look on unique values per column and number of unique values
for col in list(df.columns):
  print(col, df[col].unique(), len(df[col].unique()))

User_ID [1000001 1000002 1000003 ... 1004113 1005391 1001529] 5891
Product_ID ['P00069042' 'P00248942' 'P00087842' ... 'P00370293' 'P00371644'
 'P00370853'] 3631
Gender ['F' 'M'] 2
Age ['0-17' '55+' '26-35' '46-50' '51-55' '36-45' '18-25'] 7
Occupation [10 16 15  7 20  9  1 12 17  0  3  4 11  8 19  2 18  5 14 13  6] 21
City_Category ['A' 'C' 'B'] 3
Stay_In_Current_City_Years ['2' '4+' '3' '1' '0'] 5
Marital_Status [0 1] 2
Product_Category_1 [ 3  1 12  8  5  4  2  6 14 11 13 15  7 16 18 10 17  9 20 19] 20
Product_Category_2 [ 0.  6. 14.  2.  8. 15. 16. 11.  5.  3.  4. 12.  9. 10. 17. 13.  7. 18.] 18
Product_Category_3 [ 0. 14. 17.  5.  4. 16. 15.  8.  9. 13.  6. 12.  3. 18. 11. 10.] 16
Purchase [ 8370 15200  1422 ...   135   123   613] 18105


Product_Category_2 and Product_Category_3 are actually integers. Let's convert them to integers

In [ ]:
df = df.astype({'Product_Category_2':'int64', 'Product_Category_3':'int64'})

In [ ]:
df.info() # sanity check

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   User_ID                     550068 non-null  int64 
 1   Product_ID                  550068 non-null  object
 2   Gender                      550068 non-null  object
 3   Age                         550068 non-null  object
 4   Occupation                  550068 non-null  int64 
 5   City_Category               550068 non-null  object
 6   Stay_In_Current_City_Years  550068 non-null  object
 7   Marital_Status              550068 non-null  int64 
 8   Product_Category_1          550068 non-null  int64 
 9   Product_Category_2          550068 non-null  int64 
 10  Product_Category_3          550068 non-null  int64 
 11  Purchase                    550068 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 50.4+ MB


Other attributes' data type won't be changed for visualization but will be changed for data prep for ML

# Prepocessing

In [ ]:
# let's drop User_ID and Product_ID
df_app = df.drop(columns=['User_ID', 'Product_ID'], axis=1)
df_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Gender                      550068 non-null  object
 1   Age                         550068 non-null  object
 2   Occupation                  550068 non-null  int64 
 3   City_Category               550068 non-null  object
 4   Stay_In_Current_City_Years  550068 non-null  object
 5   Marital_Status              550068 non-null  int64 
 6   Product_Category_1          550068 non-null  int64 
 7   Product_Category_2          550068 non-null  int64 
 8   Product_Category_3          550068 non-null  int64 
 9   Purchase                    550068 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 42.0+ MB


* Convert to Integer
 1. Age
 2. Stay_In_Current_Years
 

* Hot Encode
 1. Gender
 2. City Category

In [ ]:
df_app['Age'].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [ ]:
df_app['Age'] = df_app['Age'].replace(['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'], [0, 1, 2, 3, 4, 5, 6])

In [ ]:
df_app['Age'].unique()

array([0, 6, 2, 4, 5, 3, 1])

In [ ]:
df_app['Stay_In_Current_City_Years'].unique()

array(['2', '4+', '3', '1', '0'], dtype=object)

In [ ]:
df_app['Stay_In_Current_City_Years'] = df_app['Stay_In_Current_City_Years'].replace('4+', '4')
df_app['Stay_In_Current_City_Years'] = df_app['Stay_In_Current_City_Years'].astype('int64')

In [ ]:
df_app['Stay_In_Current_City_Years'].unique()

array([2, 4, 3, 1, 0])

In [ ]:
df_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Gender                      550068 non-null  object
 1   Age                         550068 non-null  int64 
 2   Occupation                  550068 non-null  int64 
 3   City_Category               550068 non-null  object
 4   Stay_In_Current_City_Years  550068 non-null  int64 
 5   Marital_Status              550068 non-null  int64 
 6   Product_Category_1          550068 non-null  int64 
 7   Product_Category_2          550068 non-null  int64 
 8   Product_Category_3          550068 non-null  int64 
 9   Purchase                    550068 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 42.0+ MB


# Training and Prediction

In [ ]:
df_app['Gender'] = df_app['Gender'].replace(['M','F'], [0,1])
df_app['City_Category'] = df_app['City_Category'].replace(['A','B', 'C'], [0,1,2])

In [ ]:
X = df_app.drop(columns=['Purchase'])
y = df_app['Purchase']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()
model.fit(X_train, y_train)
y_preds = model.predict(X_val)

# RMSE
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_val, y_preds))

2999.920490442957

In [ ]:
import xgboost as xgb

model1 = xgb.XGBRegressor(n_jobs=-1, verbose=2)
model1.fit(X_train, y_train)
y_preds1 = model1.predict(X_val)

# RMSE
np.sqrt(mean_squared_error(y_val, y_preds1))

[07:23:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


2999.5689530805976

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(n_jobs=-1, verbose=2)
model2.fit(X_train, y_train)
y_preds2 = model2.predict(X_val)

# RMSE
np.sqrt(mean_squared_error(y_val, y_preds2))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   35.7s


building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.6min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    1.2s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    3.2s finished


3063.3746252798037

In [27]:
from sklearn.model_selection import cross_val_score

xgbreg = xgb.XGBRegressor()
score = cross_val_score(xgbreg, X, y, cv = 10, n_jobs=-1, scoring='neg_mean_squared_error', verbose=3)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.0min finished


In [31]:
pred = np.sqrt(-score)
pred

array([2976.09834546, 3005.49608974, 2999.59622195, 2971.91042085,
       2993.89044188, 2984.69928124, 2981.04922551, 3000.57283965,
       2999.55114869, 3015.90478107])